<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Edge_Detection_and_Laplacian_Addis_Abbaba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DRZIjlqbfRPxYRUA9LZ86iJDj__bMX2rmo3DMCuVHDs&tc=qi_jV5CM7G_LCZojky9wpnva6EvhtUfaxdzyY5kHh2o&cc=EIeFttVi4KGWyKADT_Lju4so0Q_Wh8aS6f5QsdwSMtY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmkCcGgQyco2KdNwZAfWT4j0Z4H41pEEGSGeNrzHHzcgH1a5DMCZWs

Successfully saved authorization token.


In [38]:
# Edge Detection

#Load point
point = ee.Geometry.Point([38.7496, 9.006])

# Compute buffer of the point
region = point.buffer(10000)

In [39]:
# Load Landsa data
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_168054_20160209')

# Define the Laplacian or edge detec tion kernel
lapacian = ee.Kernel.laplacian8(**{
    'normalize': False
})

# Apply the edge detection kernel
edgeImage = image.convolve(laplacian)

In [41]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[9, 39], zoom_start = 10)

# Add the layer to the map object.
my_map.add_ee_layer(edgeImage.clip(region), {'bands': ['B5', 'B4', 'B3'], 'max': 0.5, 'format': 'png'},'Edge Image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)